FIPS: areas in the united states. 10 digits 
First two numbers in the state, nest three: county, 5: track 
	Track is defined by population, if lots of people subdivide the county by population size. 

For the data dictionary, pickle 
1.	Want to find the unique state fips in the data dictionary, 
a.	Just state code (two digits) 
2.	Get rid of the _long  tables in my data dictionary 
3.	Split the data dictionary so that three dictionaries 
a.	one has only county level data
b.	tract level data
c.	other level (block) 	
4.	no index (just numbers) 
a.	.reset_index()
i.	Index goes to column
5.	FIPS has to be integer not string 


In [2]:
import pandas as pd
import pickle
import numpy as np

d= pickle.load(open("./dataset.pickle", "rb"))


1. Want to find the unique state fips in the data dictionary,

In [5]:
states=[]
for keys in d:
    if keys not in ['facilities_and_providers', "broadband_speeds"]:
        me =d[keys]
        me= me.reset_index()
        states=np.append(states, [item[0:2] for item in me.FIPS])
        states= np.unique(states)
print(states)   



['33' '50']


2.	Get rid of the _long  tables in my data dictionary 

In [6]:
d_no_long = {keys:values  for keys, values in d.items() if "_long" not in keys}
type(d_no_long["rf_and_screening_county"].index)


pandas.core.indexes.multi.MultiIndex

3.	Split the data dictionary so that three dictionaries 
a.	one has only county level data
b.	tract level data
c.	other level (block) 

In [7]:
county={}
#county = {keys:values for keys, values in d.items()  if (keys in ['facilities_and_providers', "broadband_speeds"])& (len(values.reset_index().FIPS[1])==5)}
track={}
others = {keys:values for keys, values in d.items() if keys in ['facilities_and_providers', "broadband_speeds"]}

for keys, values in d.items():
    if keys not in ['facilities_and_providers', "broadband_speeds"]:
        me = values
        me = me.reset_index()
        if len(me.FIPS[1])==5:
            county.update({keys:values})
        else:
            track.update({keys:values})
print(county)

{'rf_and_screening_county': measure                   BMI_Obese  Cancer_Prevalence  Currently_Smoke  \
FIPS  County       State                                                  
33001 Belknap      NH         0.308              0.091            0.157   
33003 Carroll      NH         0.294              0.100            0.156   
33005 Cheshire     NH         0.300              0.079            0.164   
33007 Coos         NH         0.343              0.088            0.200   
33009 Grafton      NH         0.265              0.084            0.149   
33011 Hillsborough NH         0.325              0.074            0.152   
33013 Merrimack    NH         0.302              0.081            0.152   
33015 Rockingham   NH         0.280              0.080            0.139   
33017 Strafford    NH         0.306              0.069            0.155   
33019 Sullivan     NH         0.317              0.089            0.183   
50001 Addison      VT         0.246              0.079            0.137 

In [8]:
track 

{'rf_and_screening_tract': measure                    BMI_Obese  Cancer_Prevalence  Currently_Smoke  \
 FIPS        County  State                                                  
 33001965100 Belknap NH         0.308              0.084            0.154   
 33001965200 Belknap NH         0.318              0.089            0.158   
 33001965300 Belknap NH         0.297              0.095            0.135   
 33001965400 Belknap NH         0.310              0.080            0.155   
 33001965598 Belknap NH         0.310              0.087            0.176   
 ...                              ...                ...              ...   
 50027966200 Windsor VT         0.303              0.082            0.184   
 50027966300 Windsor VT         0.295              0.090            0.178   
 50027966500 Windsor VT         0.281              0.088            0.156   
 50027966600 Windsor VT         0.302              0.085            0.199   
 50027966700 Windsor VT         0.297             

In [9]:
others

{'broadband_speeds':        StateAbbr        BlockCode  MaxAdDown  MaxAdUp
 0             NH  330151071001002        0.0      0.0
 1             NH  330130030012038        0.0      0.0
 2             NH  330112004002063        0.0      0.0
 3             NH  330019660004031        0.0      0.0
 4             NH  330199759013025        0.0      0.0
 ...          ...              ...        ...      ...
 700141        NH  330199759024008        2.0      1.3
 700142        NH  330199759024009        2.0      1.3
 700143        NH  330199759024010        2.0      1.3
 700144        NH  330199759024011        2.0      1.3
 700145        NH  330199759024012        2.0      1.3
 
 [700146 rows x 4 columns],
 'facilities_and_providers':                         Type                        Name  \
 413   Colon & Rectal Surgeon             JEREMY DRESSLER   
 416   Colon & Rectal Surgeon                SAMUEL LABOW   
 415   Colon & Rectal Surgeon                KRISTA EVANS   
 414   Colon & Rec

4.	no index (just numbers) 

In [10]:
no_index = { keys:values.reset_index() for keys, values in d.items() }
no_index
#there are some that have index as just numbers, fix to check if there isnt an index and we reset

{'rf_and_screening_county': measure   FIPS        County State  BMI_Obese  Cancer_Prevalence  \
 0        33001       Belknap    NH      0.308              0.091   
 1        33003       Carroll    NH      0.294              0.100   
 2        33005      Cheshire    NH      0.300              0.079   
 3        33007          Coos    NH      0.343              0.088   
 4        33009       Grafton    NH      0.265              0.084   
 5        33011  Hillsborough    NH      0.325              0.074   
 6        33013     Merrimack    NH      0.302              0.081   
 7        33015    Rockingham    NH      0.280              0.080   
 8        33017     Strafford    NH      0.306              0.069   
 9        33019      Sullivan    NH      0.317              0.089   
 10       50001       Addison    VT      0.246              0.079   
 11       50003    Bennington    VT      0.255              0.090   
 12       50005     Caledonia    VT      0.289              0.084   
 13    

5.	FIPS has to be integer not string 

In [48]:
for keys, values in no_index.items():
    if keys not in ['facilities_and_providers', "broadband_speeds"]:
        values=values.astype({"FIPS":"int64"})
        print(values.dtypes)


measure
FIPS                     int64
County                  object
State                   object
BMI_Obese              float64
Cancer_Prevalence      float64
Currently_Smoke        float64
Met_Breast_Screen      float64
Met_Cervical_Screen    float64
Met_Colon_Screen       float64
dtype: object
index        int64
FIPS         int64
County      object
State       object
measure     object
value      float64
dtype: object
measure
FIPS                   int64
County                object
State                 object
BMI_Obese            float64
Cancer_Prevalence    float64
Currently_Smoke      float64
Met_Breast_Screen    float64
Met_Colon_Screen     float64
dtype: object
index        int64
FIPS         int64
County      object
State       object
measure     object
value      float64
dtype: object
Site
FIPS                            int64
County                         object
State                          object
Type                           object
All Site                      fl

In [29]:
#better to do a function here, and call function inside 
#FIPS_int = {keys:values.astype({"FIPS":"int"}) for keys, values in no_index.items() if keys not in ['facilities_and_providers', "broadband_speeds"]}
#print(FIPS_int)


In [92]:
#file path "./dataset.pickle", "rb"
class FIPSdata: 
    def __init__(self, filepath):
        self.data= pickle.load(open(filepath, "rb"))
        self.data= {key:value for key, value in self.data.items() if "_long" not in key}
        self.remove_index()
        
    def remove_index(self):
        data=self.data
        for key, value in data.items():
            newData= values.reset_index()
            if newData.columns.str.contains("index").sum():
                #newData.drop("index", axis=1, inplace=T)
                pass
            else:
                self.data[key]=newData        
    @property             
    def states(self):
        states=[]
        for keys,values in self.data.items():
            if keys not in ['facilities_and_providers', "broadband_speeds"]:
                values.reset_index(inplace=True)
                states=np.append(states, [item[0:2] for item in values.FIPS])
                states= np.unique(states)
        self._states=states
        return(states)
    
    def Fips_int(self):
        for keys, values in self.data.items():
            if keys not in ['facilities_and_providers', "broadband_speeds"]:
                self.data[keys]=values.astype({"FIPS":"int64"})
    def split_level(self):
        self.county={}
        self.track={}
        self.others = {keys:values for keys, values in d.items() if keys in ['facilities_and_providers', "broadband_speeds"]}

        for keys, values in self.data.items():
            if keys not in ['facilities_and_providers', "broadband_speeds"]:
                if len(str(values.FIPS[1]))==5:
                    self.county.update({keys:values})
                else:
                    self.track.update({keys:values})


In [84]:
fips= FIPSdata("./dataset.pickle")
fips.data.keys()


dict_keys(['rf_and_screening_county', 'rf_and_screening_tract', 'cancer_incidence', 'cancer_mortality', 'economy_county', 'economy_tract', 'ht_county', 'ht_tract', 'sociodemographics_county', 'sociodemographics_tract', 'environment_county', 'environment_tract', 'broadband_speeds', 'facilities_and_providers'])

In [85]:
fips.states

array(['33', '50'], dtype='<U32')

In [86]:
fips._states


array(['33', '50'], dtype='<U32')

In [87]:
#finish the last two as methods in class 

In [88]:
fips.Fips_int()
fips.data


{'rf_and_screening_county': measure   FIPS        County State  BMI_Obese  Cancer_Prevalence  \
 0        33001       Belknap    NH      0.308              0.091   
 1        33003       Carroll    NH      0.294              0.100   
 2        33005      Cheshire    NH      0.300              0.079   
 3        33007          Coos    NH      0.343              0.088   
 4        33009       Grafton    NH      0.265              0.084   
 5        33011  Hillsborough    NH      0.325              0.074   
 6        33013     Merrimack    NH      0.302              0.081   
 7        33015    Rockingham    NH      0.280              0.080   
 8        33017     Strafford    NH      0.306              0.069   
 9        33019      Sullivan    NH      0.317              0.089   
 10       50001       Addison    VT      0.246              0.079   
 11       50003    Bennington    VT      0.255              0.090   
 12       50005     Caledonia    VT      0.289              0.084   
 13    

In [90]:
fips.split_level()
fips.county


{'rf_and_screening_county': measure  level_0  index   FIPS        County State  BMI_Obese  \
 0              0      0  33001       Belknap    NH      0.308   
 1              1      1  33003       Carroll    NH      0.294   
 2              2      2  33005      Cheshire    NH      0.300   
 3              3      3  33007          Coos    NH      0.343   
 4              4      4  33009       Grafton    NH      0.265   
 5              5      5  33011  Hillsborough    NH      0.325   
 6              6      6  33013     Merrimack    NH      0.302   
 7              7      7  33015    Rockingham    NH      0.280   
 8              8      8  33017     Strafford    NH      0.306   
 9              9      9  33019      Sullivan    NH      0.317   
 10            10     10  50001       Addison    VT      0.246   
 11            11     11  50003    Bennington    VT      0.255   
 12            12     12  50005     Caledonia    VT      0.289   
 13            13     13  50007    Chittenden    

In [91]:
fips.others


{'broadband_speeds':        StateAbbr        BlockCode  MaxAdDown  MaxAdUp
 0             NH  330151071001002        0.0      0.0
 1             NH  330130030012038        0.0      0.0
 2             NH  330112004002063        0.0      0.0
 3             NH  330019660004031        0.0      0.0
 4             NH  330199759013025        0.0      0.0
 ...          ...              ...        ...      ...
 700141        NH  330199759024008        2.0      1.3
 700142        NH  330199759024009        2.0      1.3
 700143        NH  330199759024010        2.0      1.3
 700144        NH  330199759024011        2.0      1.3
 700145        NH  330199759024012        2.0      1.3
 
 [700146 rows x 4 columns],
 'facilities_and_providers':                         Type                        Name  \
 413   Colon & Rectal Surgeon             JEREMY DRESSLER   
 416   Colon & Rectal Surgeon                SAMUEL LABOW   
 415   Colon & Rectal Surgeon                KRISTA EVANS   
 414   Colon & Rec